# Домашнее задание  № 5. Матричные разложения/Тематическое моделирование

### Задание № 1 (4 балла)

Попробуйте матричные разложения с 5 классификаторами - SGDClassifier, KNeighborsClassifier,  RandomForest, ExtraTreesClassifier (про него подробнее почитайте в документации, он похож на RF). Используйте и NMF и SVD. Сравните результаты на кросс-валидации и выберите лучшее сочетание.

В итоге у вас должно получиться, как минимум 10 моделей (два разложения на каждый классификатор). Используйте 1 и те же параметры кросс-валидации. Параметры векторизации, параметры K в матричных разложениях, параметры классификаторов могут быть разными между экспериментами.

Можете взять поменьше данных, если все будет обучаться слишком долго (не ставьте параметр K слишком большим в NMF, иначе точно будет слишком долго)

In [4]:
!pip install pymorphy2
!pip install pyLDAvis
!pip install razdel
import gensim
import pandas as pd
import numpy as np
from pandas import Series
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim_models
from collections import Counter
from string import punctuation
from razdel import tokenize as razdel_tokenize
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.decomposition import TruncatedSVD, NMF, PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
morph = MorphAnalyzer()
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = ins

In [5]:
# функция для лемматизации текста
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving avito_category_classification.csv to avito_category_classification.csv


In [ ]:
data = pd.read_csv('avito_category_classification.csv')
data['description_norm'] = data['description'].apply(normalize)

In [ ]:
def eval_table(X, y, pipeline, N=6):
    # зафиксируем порядок классов
    labels = list(set(y))
    
    # метрики отдельных фолдов и ошибок
    fold_metrics = pd.DataFrame(index=labels)
    errors = np.zeros((len(labels), len(labels)))
    
    # создаем стратегию кросс-валидации
    kfold = StratifiedKFold(n_splits=N, shuffle=True, )
    
    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])
        
        # записываем метрику и индекс фолда
        fold_metrics[f'precision_{i}'] = precision_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'recall_{i}'] = recall_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels, normalize='true')
    
    # таблица для усредненных значений
    result = pd.DataFrame(index=labels)
    result['precision'] = fold_metrics[[f'precision_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['precision_std'] = fold_metrics[[f'precision_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['recall'] = fold_metrics[[f'recall_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['recall_std'] = fold_metrics[[f'recall_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['f1_std'] = fold_metrics[[f'f1_{i}' for i in range(N)]].std(axis=1).round(2)
    
    # добавим одну колонку со средним по всем классам
    result.loc['mean'] = result.mean().round(2)
    # проценты ошибок просто усредняем
    errors /= N
    
    return result, errors

In [ ]:
# пайплайны для матричного разложения с SGDClassifier
pipeline_svd_sgdc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', SGDClassifier())
])

In [ ]:
pipeline_nmf_sgdc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(100)),
    ('clf', SGDClassifier())
])

In [ ]:
metrics_svd_sgdc, errors_svd_sgdc = eval_table(data['description_norm'], data['category_name'], pipeline_svd_sgdc)

In [ ]:
pd.set_option('display.max_columns', None)
print (metrics_svd_sgdc)

                            precision  precision_std  recall  recall_std  \
Детская одежда и обувь           0.74           0.02    0.78        0.03   
Квартиры                         0.96           0.03    0.96        0.02   
Товары для детей и игрушки       0.72           0.06    0.63        0.03   
Телефоны                         0.79           0.09    0.82        0.02   
Бытовая техника                  0.66           0.05    0.52        0.05   
Мебель и интерьер                0.70           0.04    0.63        0.02   
Ремонт и строительство           0.56           0.06    0.48        0.03   
Предложение услуг                0.75           0.05    0.75        0.02   
Автомобили                       0.86           0.01    0.90        0.04   
Одежда, обувь, аксессуары        0.73           0.03    0.76        0.02   
mean                             0.75           0.04    0.72        0.03   

                              f1  f1_std  
Детская одежда и обувь      0.76    0.01  
К

In [ ]:
metrics_nmf_sgdc, errors_nmf_sgdc = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_sgdc)

In [ ]:
pd.set_option('display.max_columns', None)
print (metrics_nmf_sgdc)

                            precision  precision_std  recall  recall_std  \
Детская одежда и обувь           0.54           0.11    0.73        0.08   
Квартиры                         0.72           0.11    0.95        0.04   
Товары для детей и игрушки       0.84           0.12    0.30        0.07   
Телефоны                         0.67           0.14    0.60        0.16   
Бытовая техника                  0.61           0.38    0.11        0.09   
Мебель и интерьер                0.74           0.19    0.23        0.20   
Ремонт и строительство           0.19           0.16    0.19        0.24   
Предложение услуг                0.66           0.21    0.25        0.18   
Автомобили                       0.63           0.19    0.80        0.21   
Одежда, обувь, аксессуары        0.67           0.07    0.62        0.12   
mean                             0.63           0.17    0.48        0.14   

                              f1  f1_std  
Детская одежда и обувь      0.61    0.06  
К

In [ ]:
# пайплайны для матричного разложения с KNeighborsClassifier
pipeline_svd_knc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', KNeighborsClassifier())
])

In [ ]:
pipeline_nmf_knc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(100)),
    ('clf', KNeighborsClassifier())
])

In [ ]:
metrics_svd_knc, errors_svd_knc = eval_table(data['description_norm'], data['category_name'], pipeline_svd_knc)

In [ ]:
pd.set_option('display.max_columns', None)
print (metrics_svd_knc)

                            precision  precision_std  recall  recall_std  \
Детская одежда и обувь           0.47           0.01    0.62        0.03   
Квартиры                         0.92           0.03    0.72        0.04   
Товары для детей и игрушки       0.52           0.04    0.26        0.04   
Телефоны                         0.69           0.08    0.30        0.03   
Бытовая техника                  0.25           0.03    0.25        0.05   
Мебель и интерьер                0.37           0.06    0.28        0.04   
Ремонт и строительство           0.27           0.07    0.17        0.04   
Предложение услуг                0.59           0.05    0.55        0.03   
Автомобили                       0.53           0.04    0.60        0.04   
Одежда, обувь, аксессуары        0.47           0.02    0.57        0.02   
mean                             0.51           0.04    0.43        0.04   

                              f1  f1_std  
Детская одежда и обувь      0.53    0.01  
К

In [ ]:
metrics_nmf_knc, errors_nmf_knc = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_knc)

In [ ]:
pd.set_option('display.max_columns', None)
print (metrics_nmf_knc)

                            precision  precision_std  recall  recall_std  \
Детская одежда и обувь           0.57           0.03    0.54        0.03   
Квартиры                         0.86           0.07    0.81        0.05   
Товары для детей и игрушки       0.48           0.04    0.34        0.03   
Телефоны                         0.63           0.03    0.46        0.05   
Бытовая техника                  0.20           0.04    0.28        0.04   
Мебель и интерьер                0.39           0.06    0.31        0.05   
Ремонт и строительство           0.26           0.07    0.22        0.05   
Предложение услуг                0.44           0.02    0.69        0.05   
Автомобили                       0.53           0.09    0.72        0.06   
Одежда, обувь, аксессуары        0.54           0.02    0.55        0.02   
mean                             0.49           0.05    0.49        0.04   

                              f1  f1_std  
Детская одежда и обувь      0.55    0.02  
К

In [ ]:
# пайплайны для матричного разложения с RandomForest
pipeline_svd_rfc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', RandomForestClassifier())
])

In [ ]:
pipeline_nmf_rfc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(100)),
    ('clf', RandomForestClassifier())
])

In [ ]:
metrics_svd_rfc, errors_svd_rfc = eval_table(data['description_norm'], data['category_name'], pipeline_svd_rfc)

In [ ]:
pd.set_option('display.max_columns', None)
print (metrics_svd_rfc)

                            precision  precision_std  recall  recall_std  \
Детская одежда и обувь           0.48           0.02    0.71        0.01   
Квартиры                         0.86           0.03    0.90        0.02   
Товары для детей и игрушки       0.66           0.07    0.27        0.01   
Телефоны                         0.87           0.06    0.45        0.07   
Бытовая техника                  0.64           0.22    0.12        0.04   
Мебель и интерьер                0.72           0.10    0.13        0.05   
Ремонт и строительство           0.67           0.14    0.13        0.04   
Предложение услуг                0.77           0.06    0.54        0.04   
Автомобили                       0.81           0.04    0.66        0.03   
Одежда, обувь, аксессуары        0.51           0.01    0.74        0.02   
mean                             0.70           0.08    0.46        0.03   

                              f1  f1_std  
Детская одежда и обувь      0.58    0.02  
К

In [ ]:
metrics_nmf_rfc, errors_nmf_rfc = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_rfc)

In [ ]:
pd.set_option('display.max_columns', None)
print (metrics_nmf_rfc)

                            precision  precision_std  recall  recall_std  \
Детская одежда и обувь           0.72           0.01    0.78        0.03   
Квартиры                         0.92           0.02    0.97        0.01   
Товары для детей и игрушки       0.75           0.02    0.60        0.04   
Телефоны                         0.79           0.03    0.78        0.04   
Бытовая техника                  0.70           0.10    0.33        0.07   
Мебель и интерьер                0.72           0.05    0.56        0.05   
Ремонт и строительство           0.62           0.05    0.47        0.04   
Предложение услуг                0.70           0.04    0.70        0.06   
Автомобили                       0.83           0.03    0.89        0.02   
Одежда, обувь, аксессуары        0.69           0.02    0.78        0.02   
mean                             0.74           0.04    0.69        0.04   

                              f1  f1_std  
Детская одежда и обувь      0.75    0.01  
К

In [ ]:
# пайплайны для матричного разложения с ExtraTreesClassifier
pipeline_svd_etc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', ExtraTreesClassifier())
])

In [ ]:
pipeline_nmf_etc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(100)),
    ('clf', ExtraTreesClassifier())
])

In [ ]:
metrics_svd_etc, errors_svd_etc = eval_table(data['description_norm'], data['category_name'], pipeline_svd_etc)

In [ ]:
pd.set_option('display.max_columns', None)
print (metrics_svd_etc)

                            precision  precision_std  recall  recall_std  \
Детская одежда и обувь           0.44           0.02    0.71        0.02   
Квартиры                         0.71           0.02    0.78        0.04   
Товары для детей и игрушки       0.65           0.07    0.17        0.03   
Телефоны                         0.91           0.08    0.32        0.08   
Бытовая техника                  0.60           0.20    0.15        0.04   
Мебель и интерьер                0.77           0.11    0.13        0.01   
Ремонт и строительство           0.63           0.17    0.09        0.02   
Предложение услуг                0.86           0.09    0.26        0.05   
Автомобили                       0.80           0.06    0.48        0.04   
Одежда, обувь, аксессуары        0.46           0.02    0.71        0.02   
mean                             0.68           0.08    0.38        0.03   

                              f1  f1_std  
Детская одежда и обувь      0.54    0.02  
К

In [ ]:
metrics_nmf_etc, errors_nmf_etc = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_etc)

In [ ]:
pd.set_option('display.max_columns', None)
print (metrics_nmf_etc)

                            precision  precision_std  recall  recall_std  \
Детская одежда и обувь           0.74           0.03    0.77        0.01   
Квартиры                         0.92           0.02    0.97        0.02   
Товары для детей и игрушки       0.76           0.04    0.60        0.04   
Телефоны                         0.79           0.06    0.79        0.04   
Бытовая техника                  0.70           0.08    0.30        0.06   
Мебель и интерьер                0.73           0.04    0.56        0.04   
Ремонт и строительство           0.61           0.07    0.48        0.07   
Предложение услуг                0.70           0.03    0.75        0.03   
Автомобили                       0.84           0.02    0.91        0.03   
Одежда, обувь, аксессуары        0.69           0.02    0.80        0.03   
mean                             0.75           0.04    0.69        0.04   

                              f1  f1_std  
Детская одежда и обувь      0.76    0.02  
К

In [ ]:
# сравним метрики по среднему
metrics = pd.DataFrame(columns = metrics_svd_sgdc.columns)

metrics = metrics.append(metrics_svd_sgdc.loc['mean'])
metrics = metrics.append(metrics_nmf_sgdc.loc['mean'])
metrics = metrics.append(metrics_svd_knc.loc['mean'])
metrics = metrics.append(metrics_nmf_knc.loc['mean'])
metrics = metrics.append(metrics_svd_rfc.loc['mean'])
metrics = metrics.append(metrics_nmf_rfc.loc['mean'])
metrics = metrics.append(metrics_svd_etc.loc['mean'])
metrics = metrics.append(metrics_nmf_etc.loc['mean'])

metrics.index = Series (['mean_svd_sgdc', 'mean_nmf_sgdc', 'mean_svd_knc', 'mean_nmf_knc', 'mean_svd_rfc', 'mean_nmf_rfc', 'mean_svd_etc', 'mean_nmf_etc'])

In [ ]:
print (metrics)

               precision  precision_std  recall  recall_std    f1  f1_std
mean_svd_sgdc       0.75           0.04    0.72        0.03  0.73    0.02
mean_nmf_sgdc       0.63           0.17    0.48        0.14  0.46    0.11
mean_svd_knc        0.51           0.04    0.43        0.04  0.45    0.03
mean_nmf_knc        0.49           0.05    0.49        0.04  0.48    0.04
mean_svd_rfc        0.70           0.08    0.46        0.03  0.50    0.04
mean_nmf_rfc        0.74           0.04    0.69        0.04  0.70    0.03
mean_svd_etc        0.68           0.08    0.38        0.03  0.42    0.04
mean_nmf_etc        0.75           0.04    0.69        0.04  0.71    0.03


Лучше всего работает классификатор SGD с разложением SVD. Если сравнивать по F1 мере, то высокие значения также у ExtraTreesClassifier с NMF и RandomForest с NMF. Параметры у классификаторов дефолтные, поэтому при их настройке соотношение может поменяться, однако у трех выделенных классификаторов большой отрыв.
Точность классификации сильно страдает у SGD с NMF.

### Задание № 2 (6 баллов)

В Gensim тоже можно добавить нграммы и tfidf. Постройте 1 модель без них (как в семинаре) и еще 3 модели (1 с нграммами, 1 с tfidf и 1 с нграммами и с tfidf). Сранивте качество с помощью метрик (перплексия, когерентность) и на глаз. Определите лучшую модель. Для каждой модели выберите 1 самую красивую на ваш взгляд тему.

Используйте данные википедии из семинара. Можете взять поменьше данных, если все обучается долго.

Важное требование - получившиеся модели не должны быть совсем плохими. Если хороших тем не получается, попробуйте настроить гиперпараметры, отфильтровать словарь по-другому. 

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving wiki_data.txt to wiki_data.txt


In [6]:
texts = open('wiki_data.txt').read().splitlines()[:1000]
texts = ([normalize(text) for text in texts])

In [8]:
#  обычная модель без нграмм и tfidf
dictionary = gensim.corpora.Dictionary((text.split() for text in texts))
dictionary.filter_extremes(no_above=0.1, no_below=10)
dictionary.compactify()

corpus = [dictionary.doc2bow(text.split()) for text in texts]

In [9]:
lda = gensim.models.LdaMulticore(corpus, 
                                 100,
                                 alpha='asymmetric',
                                 id2word=dictionary, 
                                 passes=10)

In [10]:
lda.print_topics()

[(99,
  '0.043*"россия" + 0.034*"экономический" + 0.029*"общество" + 0.015*"российский" + 0.010*"организация" + 0.009*"деятельность" + 0.008*"член" + 0.008*"развитие" + 0.008*"номер" + 0.008*"общественный"'),
 (98,
  '0.018*"школа" + 0.018*"газета" + 0.018*"современный" + 0.015*"журнал" + 0.012*"московский" + 0.012*"б" + 0.011*"россия" + 0.010*"наука" + 0.009*"сын" + 0.009*"изображение"'),
 (96,
  '0.033*"эксперимент" + 0.016*"проект" + 0.016*"исследование" + 0.015*"полёт" + 0.012*"состояние" + 0.011*"экипаж" + 0.011*"пространство" + 0.011*"система" + 0.010*"этап" + 0.009*"условие"'),
 (97,
  '0.043*"э" + 0.042*"н" + 0.036*"рим" + 0.030*"италия" + 0.020*"итальянский" + 0.015*"сын" + 0.014*"л" + 0.013*"римский" + 0.013*"соглашение" + 0.012*"объявить"'),
 (95,
  '0.025*"переход" + 0.021*"уровень" + 0.016*"температура" + 0.014*"точка" + 0.014*"энергия" + 0.013*"свет" + 0.010*"выставка" + 0.010*"помощь" + 0.009*"скорость" + 0.009*"реконструкция"'),
 (93,
  '0.064*"канада" + 0.031*"сельсове

In [163]:
print (texts [:10])

['новостройка нижегородский область новостро́йка — сельский посёлок в дивеевский район нижегородский область входить в состав сатисский сельсовет посёлок расположить в 12,5 км к юг от село дивеево и 1 км к запад от город саров на право берег река вичкинза правый приток река сатис окружить смешанный лес соединить асфальтовый дорога с посёлок цыгановка 1,5 км и грунтовый просёлочный дорога с посёлок сатис 3,5 км название новостройка являться сугубо официальный местный население использовать исключительно альтернативный название — хитрый употребляться языковой оборот « … на хитрый » ранее использовать название — песчаный известковый основать в 1920-й год переселенец из соседний село аламасовый и нарышкино расположить соответственно в 8 и 14 км к запад в вознесенский район традиционно в посёлок жить рабочий совхоз « вперёд » центр в посёлок сатис возле посёлок расположить карьер где активно добывать доломитовый мука и бутовый камень в настоящий время официально закрытый по данные 1978 год 

In [11]:
# модель с нграммами
texts_splitted = [text.split() for text in texts]
ph = gensim.models.Phrases(texts_splitted, scoring='npmi', threshold=0.4)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = [text for text in p[texts_splitted]]

dictionary_ngrammed = gensim.corpora.Dictionary(ngrammed_texts)
dictionary_ngrammed.filter_extremes(no_above=0.1, no_below=10)
dictionary_ngrammed.compactify()

corpus_ngrammed = [dictionary_ngrammed.doc2bow(text) for text in ngrammed_texts]

In [12]:
lda_ngrammed = gensim.models.LdaMulticore(corpus_ngrammed, 
                                          100,
                                          alpha='asymmetric',
                                          id2word=dictionary_ngrammed, 
                                          passes=10)

In [13]:
lda_ngrammed.print_topics()

[(99,
  '0.042*"выиграть" + 0.041*"гонка" + 0.026*"сезон" + 0.026*"победа" + 0.020*"раз" + 0.018*"круг" + 0.013*"500" + 0.012*"команда" + 0.012*"подряд" + 0.011*"следующий"'),
 (98,
  '0.047*"департамент" + 0.035*"аргентина" + 0.031*"2001_год" + 0.025*"административный_центр" + 0.024*"муниципалитет" + 0.024*"км²" + 0.023*"плотность_население" + 0.023*"2010_год" + 0.023*"по_данные" + 0.023*"чел_км²"'),
 (97,
  '0.037*"кладбище" + 0.020*"проект" + 0.019*"городской" + 0.017*"памятник" + 0.014*"установить" + 0.013*"г" + 0.013*"выставка" + 0.012*"крест" + 0.012*"фестиваль" + 0.010*"собор"'),
 (96,
  '0.042*"животное" + 0.018*"корпус" + 0.015*"форма" + 0.014*"исследование" + 0.013*"корабль" + 0.013*"учёный" + 0.012*"связь" + 0.011*"изучение" + 0.011*"вид" + 0.010*"соединение"'),
 (94,
  '0.033*"мотив" + 0.029*"фраза" + 0.023*"музыкальный" + 0.017*"предложение" + 0.014*"король" + 0.013*"музыка" + 0.013*"сын" + 0.012*"длинный" + 0.010*"юг" + 0.010*"альбом"'),
 (95,
  '0.049*"проект" + 0.042*"з

In [14]:
#  модель с tfidf
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary, )
corpus_tfidf = tfidf[corpus]

In [15]:
lda_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                          100,
                                          alpha='asymmetric',
                                          id2word=dictionary, 
                                          passes=10)

In [16]:
lda_tfidf.print_topics()

[(99,
  '0.000*"конечный" + 0.000*"круглый" + 0.000*"изобразить" + 0.000*"историк" + 0.000*"каменный" + 0.000*"комиссар" + 0.000*"конгресс" + 0.000*"здание" + 0.000*"красивый" + 0.000*"лишить"'),
 (98,
  '0.000*"конечный" + 0.000*"круглый" + 0.000*"изобразить" + 0.000*"историк" + 0.000*"каменный" + 0.000*"комиссар" + 0.000*"конгресс" + 0.000*"здание" + 0.000*"красивый" + 0.000*"лишить"'),
 (97,
  '0.000*"конечный" + 0.000*"круглый" + 0.000*"изобразить" + 0.000*"историк" + 0.000*"каменный" + 0.000*"комиссар" + 0.000*"конгресс" + 0.000*"здание" + 0.000*"красивый" + 0.000*"лишить"'),
 (96,
  '0.000*"конечный" + 0.000*"круглый" + 0.000*"изобразить" + 0.000*"историк" + 0.000*"каменный" + 0.000*"комиссар" + 0.000*"конгресс" + 0.000*"здание" + 0.000*"красивый" + 0.000*"лишить"'),
 (94,
  '0.000*"выжить" + 0.000*"пассажир" + 0.000*"самолёт" + 0.000*"день" + 0.000*"погибнуть" + 0.000*"пилот" + 0.000*"чтобы" + 0.000*"экспедиция" + 0.000*"поход" + 0.000*"вернуться"'),
 (95,
  '0.000*"конечный" + 

In [17]:
# модель с нграммами и tfidf
corpus_ngrammed_tfidf = tfidf[corpus_ngrammed]

lda_ngrammed_tfidf = gensim.models.LdaMulticore(corpus_ngrammed_tfidf, 
                                        100,
                                        alpha='asymmetric',
                                        id2word=dictionary_ngrammed, 
                                        passes=10)

In [18]:
lda_ngrammed_tfidf.print_topics()

[(99,
  '0.000*"священник" + 0.000*"символ" + 0.000*"религиозный" + 0.000*"решение" + 0.000*"рим" + 0.000*"рядом" + 0.000*"самостоятельный" + 0.000*"свой_очередь" + 0.000*"расположение" + 0.000*"северный_часть"'),
 (98,
  '0.000*"священник" + 0.000*"символ" + 0.000*"религиозный" + 0.000*"решение" + 0.000*"рим" + 0.000*"рядом" + 0.000*"самостоятельный" + 0.000*"свой_очередь" + 0.000*"расположение" + 0.000*"северный_часть"'),
 (97,
  '0.000*"священник" + 0.000*"символ" + 0.000*"религиозный" + 0.000*"решение" + 0.000*"рим" + 0.000*"рядом" + 0.000*"самостоятельный" + 0.000*"свой_очередь" + 0.000*"расположение" + 0.000*"северный_часть"'),
 (96,
  '0.000*"священник" + 0.000*"символ" + 0.000*"религиозный" + 0.000*"решение" + 0.000*"рим" + 0.000*"рядом" + 0.000*"самостоятельный" + 0.000*"свой_очередь" + 0.000*"расположение" + 0.000*"северный_часть"'),
 (95,
  '0.001*"корабль" + 0.000*"море" + 0.000*"сила" + 0.000*"противник" + 0.000*"проект" + 0.000*"морской" + 0.000*"задача" + 0.000*"удар" + 

In [19]:
# сравнение моделей по метрикам
import numpy as np

print ('perplexion lda: {}'.format(np.exp2(-lda.log_perplexity(corpus[:1000]))))
print ('perplexion lda_ngrammed: {}'.format(np.exp2(-lda_ngrammed.log_perplexity(corpus[:1000]))))
print ('perplexion lda_tfidf: {}'.format(np.exp2(-lda_tfidf.log_perplexity(corpus[:1000]))))
print ('perplexion lda_ngrammed_tfidf: {}'.format(np.exp2(-lda_ngrammed_tfidf.log_perplexity(corpus[:1000]))))


perplexion lda: 214.40808792267347
perplexion lda_ngrammed: 872.5279775414732
perplexion lda_tfidf: 240.30509363520702
perplexion lda_ngrammed_tfidf: 422.7759770581336


In [26]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts_splitted, 
                                                   dictionary=dictionary, coherence='c_v')

In [34]:
topics = []
for topic_id, topic in lda_ngrammed.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda_ngrammed = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictionary_ngrammed, coherence='c_v')

In [28]:
topics = []
for topic_id, topic in lda_tfidf.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda_tfidf = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts_splitted, 
                                                   dictionary=dictionary, coherence='c_v')

In [35]:
topics = []
for topic_id, topic in lda_ngrammed_tfidf.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda_ngrammed_tfidf = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictionary_ngrammed, coherence='c_v')

In [36]:
print ('coherence lda: {}'.format(coherence_model_lda.get_coherence()))
print ('coherence lda_ngrammed: {}'.format(coherence_model_lda_ngrammed.get_coherence()))
print ('coherence lda_tfidf: {}'.format(coherence_model_lda_tfidf.get_coherence()))
print ('coherence lda_ngrammed_tfidf: {}'.format(coherence_model_lda_ngrammed_tfidf.get_coherence()))

coherence lda: 0.49716080117511835
coherence lda_ngrammed: 0.46809469814268545
coherence lda_tfidf: 0.44330841762101203
coherence lda_ngrammed_tfidf: 0.37353929141765646


# Тематика
На мой взгляд, при добавлении нграмм и/или tfidf тематика прослеживается хуже, самые интересные и цельные примеры у обычной модели:
* **lda**:
  (96,
  '0.033*"эксперимент" + 0.016*"проект" + 0.016*"исследование" + 0.015*"полёт" + 0.012*"состояние" + 0.011*"экипаж" + 0.011*"пространство" + 0.011*"система" + 0.010*"этап" + 0.009*"условие"')
* **lda_ngrammed**:
  (95,
  '0.049*"проект" + 0.042*"здание" + 0.038*"архитектор" + 0.032*"архитектура" + 0.029*"совместно_с" + 0.026*"дом" + 0.026*"музей" + 0.021*"институт" + 0.020*"архитектурный" + 0.020*"художественный"')
* **lda_tfidf**:
  (2,
  '0.047*"экипаж" + 0.040*"бельгия" + 0.034*"финал" + 0.033*"заезд" + 0.031*"раунд" + 0.030*"серебряный" + 0.028*"бронзовый" + 0.027*"спортсмен" + 0.021*"дисциплина" + 0.021*"1960"')
* **lda_ngrammed_tfidf**:
   (9,
  '0.053*"оборудование" + 0.005*"технологический" + 0.001*"компания" + 0.001*"производить" + 0.001*"корпус" + 0.001*"дивизия" + 0.001*"купить" + 0.001*"генерал-майор" + 0.001*"операция" + 0.000*"командование"')

# Перплексия
Ближе всего к нулю перплексия обычной модели, по этому показателю плохо проявила себя модель с нграммами.

# Когерентность
Основываясь на данной метрике, лучшей моделью является опять-таки обычная, с добавлением нграмм и/или tfidf качество тем снижается.